<a href="https://colab.research.google.com/github/hamednasr/tensorflow/blob/main/04_transfer_learning_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# transfer learning in tensorflow: feature extraction

## load data

In [1]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-02-27 10:36:23--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 142.250.98.128, 142.251.107.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   107MB/s    in 1.5s    

2022-02-27 10:36:25 (107 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
import zipfile
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

In [3]:
import os
for dirpath, dirnames,filenames in os.walk('10_food_classes_10_percent'):
  print(f'there are {len(dirnames)} folders and {len(filenames)} images in {dirpath}')

there are 2 folders and 0 images in 10_food_classes_10_percent
there are 10 folders and 0 images in 10_food_classes_10_percent/test
there are 0 folders and 250 images in 10_food_classes_10_percent/test/fried_rice
there are 0 folders and 250 images in 10_food_classes_10_percent/test/chicken_wings
there are 0 folders and 250 images in 10_food_classes_10_percent/test/pizza
there are 0 folders and 250 images in 10_food_classes_10_percent/test/steak
there are 0 folders and 250 images in 10_food_classes_10_percent/test/grilled_salmon
there are 0 folders and 250 images in 10_food_classes_10_percent/test/sushi
there are 0 folders and 250 images in 10_food_classes_10_percent/test/chicken_curry
there are 0 folders and 250 images in 10_food_classes_10_percent/test/ramen
there are 0 folders and 250 images in 10_food_classes_10_percent/test/hamburger
there are 0 folders and 250 images in 10_food_classes_10_percent/test/ice_cream
there are 10 folders and 0 images in 10_food_classes_10_percent/train


## preprocess the data

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_shape = (224, 224)
batch_size = 32

train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

train_datagen = ImageDataGenerator(rescale=1/255) 
test_datagen = ImageDataGenerator(rescale=1/255) 

print('Training images:')
train_data_10_percent = train_datagen.flow_from_directory(directory=train_dir,
                                                          target_size=image_shape,
                                                          batch_size=batch_size,
                                                          class_mode='categorical')
print('\nTest images:')
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                              target_size=image_shape,
                                              batch_size=batch_size,
                                              class_mode='categorical')

Training images:
Found 750 images belonging to 10 classes.

Test images:
Found 2500 images belonging to 10 classes.


## create a tensorboard callback

In [5]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f'saving tensorboard log files to: {log_dir}')
  return tensorboard_callback

## create models using tensorflow hub

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, optimizers

In [7]:
# two models from tensorflow hub

resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5'
efficiennet_url='https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2'

In [8]:
def create_model(model_url, num_classes = 10):
  '''
  takes a tensorflow hub URL and creates a keras sequential model withit
  
  Args:
  model_url: the URL of model from tensorflow hub
  num_classes
  '''
  
  feature_extractor_layer = hub.KerasLayer(model_url,trainable=False,
                                           name='feature_extractor_layer',
                                           input_shape=image_shape+(3,))   
  
  model = tf.keras.Sequential([
                feature_extractor_layer,

                layers.Dense(num_classes, activation= 'softmax', name='output_layer')
      ])
  
  return model

### resnet model

In [9]:
resnet_model=create_model(resnet_url,train_data_10_percent.num_classes)

In [10]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_layer (Ke  (None, 2048)             42626560  
 rasLayer)                                                       
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 42,647,050
Trainable params: 20,490
Non-trainable params: 42,626,560
_________________________________________________________________


In [11]:
resnet_model.compile(loss = 'categorical_crossentropy',
                     optimizer = optimizers.Adam(),
                     metrics = ['accuracy'])

In [12]:
resnet_model.fit(train_data_10_percent,epochs=5,
                 callbacks=[create_tensorboard_callback('callbacks','tansorboard')])

saving tensorboard log files to: callbacks/tansorboard/220227-103641
Epoch 1/5
24/24 [==============================] - 25s 217ms/step - loss: 1.8535 - accuracy: 0.4013
Epoch 2/5
24/24 [==============================] - 5s 190ms/step - loss: 0.8281 - accuracy: 0.7573
Epoch 3/5
24/24 [==============================] - 5s 191ms/step - loss: 0.5565 - accuracy: 0.8413
Epoch 4/5
24/24 [==============================] - 5s 194ms/step - loss: 0.4182 - accuracy: 0.8947
Epoch 5/5
24/24 [==============================] - 5s 203ms/step - loss: 0.3438 - accuracy: 0.9200


In [13]:
resnet_model.evaluate(test_data)

79/79 [==============================] - 17s 203ms/step - loss: 0.6545 - accuracy: 0.7796


[0.6544893383979797, 0.7796000242233276]

### efficient model

In [14]:
eff_model = create_model(efficiennet_url,train_data_10_percent.num_classes)

In [16]:
eff_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_layer (Ke  (None, 1280)             5919312   
 rasLayer)                                                       
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 5,932,122
Trainable params: 12,810
Non-trainable params: 5,919,312
_________________________________________________________________


In [18]:
eff_model.compile(loss = 'categorical_crossentropy',
                 optimizer=optimizers.Adam(),
                 metrics=['accuracy'])

In [19]:
eff_model.fit(train_data_10_percent,epochs=5,
                 callbacks=[create_tensorboard_callback('callbacks','tansorboard')])

saving tensorboard log files to: callbacks/tansorboard/220227-104349
Epoch 1/5
24/24 [==============================] - 12s 173ms/step - loss: 2.0080 - accuracy: 0.3147
Epoch 2/5
24/24 [==============================] - 4s 166ms/step - loss: 1.3282 - accuracy: 0.6720
Epoch 3/5
24/24 [==============================] - 4s 168ms/step - loss: 1.0186 - accuracy: 0.7427
Epoch 4/5
24/24 [==============================] - 4s 176ms/step - loss: 0.8533 - accuracy: 0.7827
Epoch 5/5
24/24 [==============================] - 4s 172ms/step - loss: 0.7465 - accuracy: 0.8253


In [20]:
eff_model.evaluate(test_data)

79/79 [==============================] - 15s 173ms/step - loss: 0.8074 - accuracy: 0.7572


[0.8073697686195374, 0.7572000026702881]